In [1]:
import pandas as pd

In [3]:
task1_data = pd.read_csv("/data/yingfei/cancer_data/llm_prompt_data/llm_inference_data/test_fewshot_simple_mistral.csv")
print(task1_data.shape) # zero shot

(2312, 5)


In [4]:
task1_data.head()

,cell_id,prompt,answer,output_med,output_Mis
0,ACH-000006,Think step by step and decide in a single word...,Sensitive,NaN,Sensitive. While the mutations in the cell lin...
1,ACH-000006,Think step by step and decide in a single word...,Resistant,NaN,"The mutations of the cell line are ASXL1, FLT3..."
2,ACH-000006,Think step by step and decide in a single word...,Resistant,NaN,"The mutations of the cell line are ASXL1, FLT3..."
3,ACH-000006,Think step by step and decide in a single word...,Sensitive,NaN,Sensitive (LINSITINIB is known to be sensitive...
4,ACH-000006,Think step by step and decide in a single word...,Sensitive,NaN,"The mutations of the cell line are ASXL1, FLT3..."


In [6]:
task1_data = task1_data.loc[task1_data.output_Mis.notna()].copy()
task1_data.shape

(200, 5)

In [10]:
print(task1_data.output_Mis[6]) # ??? -> manual label

Sensitive. The cell line has mutations in genes other than the drug target, so the sensitivity of the cell line to the drug depends on the specific mutations in the drug target and their impact on the drug's binding and efficacy. In this case, the drug targets SRC, ABL, and TEC, and the cell line has mutations in ASXL1, FLT3, MAP3K1, NOTCH1, TP53, and U2AF1. The mutations in ASXL1, FLT3, MAP3K1, NOTCH1, and U2AF1 are not directly related to the drug target, so their impact on drug sensitivity is uncertain. However, the mutation in TP53, which is a tumor suppressor gene, could potentially make the cell line more resistant to the drug, as TP53 mutations are often associated with drug resistance. However, the drug's mechanism of action and the specific mutations in the drug target could still result in sensitivity, so the overall assessment is uncertain and requires further investigation.


In [22]:
task1_data['prediction'] = task1_data['output_Mis'].apply(lambda x: "Sensitive" if x.split(".")[0] == "Sensitive" or x.split(" ")[0] == "Sensitive" else \
                                                        "Resistant" if x.split(".")[0] == "Resistant" or x.split(" ")[0] == "Resistant" else "None" \
                                                         if "cannot" in x else "Check")

In [23]:
task1_data.prediction.value_counts()

prediction
Sensitive    95
None         52
Check        49
Resistant     4
Name: count, dtype: int64

In [25]:
task1_data['prediction'] = task1_data['output_Mis'].apply(lambda x: "Check" if "sensitive" in x.lower() and "resistant" in x.lower() else \
                                                               "Sensitive" if "sensitive" in x.lower() else "Resistant" if "resistant" in x.lower() \
                                                               else "None")

In [26]:
task1_data.prediction.value_counts()

prediction
Sensitive    84
None         79
Check        30
Resistant     7
Name: count, dtype: int64

In [27]:
check_df = task1_data.loc[task1_data.prediction.isin(["Check"])]
for i in range(len(check_df)):
    print(check_df.index.values[i], check_df.prediction.values[i])
    print(check_df.output_Mis.values[i])
    print() 

1 Check
The mutations of the cell line are ASXL1, FLT3, MAP3K1, NOTCH1, TP53, U2AF1. Based on the provided information, it is not possible to determine with certainty whether the drug NVP-TAE684 will be sensitive or resistant to this cell line. The presence of mutations in genes that are known to be involved in the drug target pathway, such as ALK and RTK signaling, can suggest sensitivity. However, other factors such as the specific mutations and their impact on the drug target, as well as potential resistance mechanisms, should also be considered. Therefore, the drug sensitivity cannot be determined with a single word in this case.

6 Check
Sensitive. The cell line has mutations in genes other than the drug target, so the sensitivity of the cell line to the drug depends on the specific mutations in the drug target and their impact on the drug's binding and efficacy. In this case, the drug targets SRC, ABL, and TEC, and the cell line has mutations in ASXL1, FLT3, MAP3K1, NOTCH1, TP53,

In [30]:
task1_data.loc[37, 'prediction']="Sensitive" 
task1_data.loc[38, 'prediction']="Sensitive"
task1_data.loc[78, 'prediction']="Sensitive"
task1_data.loc[92, 'prediction']="Sensitive"
task1_data.loc[171, 'prediction']="Sensitive"
task1_data.loc[175, 'prediction']="Sensitive"
task1_data.loc[177, 'prediction']="Sensitive"
task1_data.loc[179, 'prediction']="Sensitive"
task1_data.loc[198, 'prediction']="Sensitive"

In [31]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
accuracy_score(task1_data.answer, task1_data.prediction)

0.23

In [32]:
precision_score(task1_data.loc[task1_data.prediction.isin(["Sensitive", "Resistant"]), 'answer'],\
                task1_data.loc[task1_data.prediction.isin(["Sensitive", "Resistant"]), 'prediction'], pos_label = "Sensitive")

0.46236559139784944

In [33]:
recall_score(task1_data.loc[task1_data.prediction.isin(["Sensitive", "Resistant"]), 'answer'],\
                task1_data.loc[task1_data.prediction.isin(["Sensitive", "Resistant"]), 'prediction'], pos_label = "Sensitive")

0.9148936170212766

In [34]:
f1_score(task1_data.loc[task1_data.prediction.isin(["Sensitive", "Resistant"]), 'answer'],\
                task1_data.loc[task1_data.prediction.isin(["Sensitive", "Resistant"]), 'prediction'], pos_label = "Sensitive")

0.6142857142857143

In [38]:
confusion_matrix(task1_data.loc[task1_data.prediction.isin(["Sensitive", "Resistant"]), 'answer'],\
                task1_data.loc[task1_data.prediction.isin(["Sensitive", "Resistant"]), 'prediction'], labels =  ["Sensitive", "Resistant"])

array([[43,  4],
       [50,  3]])